In [ ]:
# |default_exp error_correction_t5

In [ ]:
# | export
from typing import Dict

In [ ]:
# | export

def filter_max_len(example: Dict, max_len: int):
    if example["len_ocr"] <= max_len and example["len_gs"] <= max_len:
        return True
    return False

In [ ]:
# | export

def preprocess_function(examples, tokenizer):
    model_inputs = tokenizer(examples["ocr"])

    labels = tokenizer(text_target=examples["gs"])

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()